In [87]:
import lzma
import pickle
import time

import dask.dataframe as dd
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from sklearn import linear_model
from torch import optim
from torch.utils.data import DataLoader, TensorDataset
from tqdm.notebook import tqdm

In [88]:
from ares import ALL_STRUCTURES
from sc2.dicts.unit_trained_from import UNIT_TRAINED_FROM

unit_types = ALL_STRUCTURES | UNIT_TRAINED_FROM.keys()
unit_type_values = {u.value for u in unit_types}

In [89]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [90]:
def process(df: pd.DataFrame) -> dd.DataFrame:
    return (
        df.pivot_table(index=["replay_name", "game_loop", "player"], columns=["is_own", "unit_type"], values="count")
        .fillna(0)
        .astype(int)
    )

In [91]:
df_train = pd.read_parquet("../resources/datasets/scout/train.parquet")
df_test = pd.read_parquet("../resources/datasets/scout/test.parquet")

In [92]:
from itertools import product


def to_numpy(df: pd.DataFrame) -> tuple[np.ndarray, np.ndarray]:
    df = df.reset_index()
    df = df.pivot_table(
        index=["replay_name", "game_loop", "player"],
        columns=["owner", "unit_type"],
        values="count",
    )
    df = df.reindex(columns=[(o, t) for o, t in product([1, 2], unit_type_values)])
    df = df.fillna(0)
    df = df.astype(int)

    x = df.reset_index().drop(["replay_name", "player"], axis=1, level=0).to_numpy()
    y = np.hstack([x[1::2, :], x[::2, :]]).reshape(x.shape)
    return x, y

In [93]:
df_train

,index,game_loop,unit_type,player,owner,count,replay_name
0,8,1,86,1,1,1,3901860_Sharkling_PhantomBot_EphemeronAIE.SC2R...
1,9,1,86,2,2,1,3901860_Sharkling_PhantomBot_EphemeronAIE.SC2R...
2,10,1,104,1,1,12,3901860_Sharkling_PhantomBot_EphemeronAIE.SC2R...
3,11,1,104,2,2,12,3901860_Sharkling_PhantomBot_EphemeronAIE.SC2R...
4,12,1,106,1,1,1,3901860_Sharkling_PhantomBot_EphemeronAIE.SC2R...
...,...,...,...,...,...,...,...
96584,103026,6227,106,1,1,2,3901860_Sharkling_PhantomBot_EphemeronAIE.SC2R...
96585,103027,6227,106,2,2,2,3901860_Sharkling_PhantomBot_EphemeronAIE.SC2R...
96586,103028,6227,151,1,1,3,3901860_Sharkling_PhantomBot_EphemeronAIE.SC2R...
96587,103029,6227,151,1,2,3,3901860_Sharkling_PhantomBot_EphemeronAIE.SC2R...


In [94]:
x_train, y_train = to_numpy(df_train)
x_test, y_test = to_numpy(df_test)

In [95]:
model = linear_model.LinearRegression()
model.fit(x_train, y_train)
model.coef_

array([[ 1.00000000e+00, -5.06002502e-26, -2.95851633e-25, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       ...,
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00]],
      shape=(273, 273))

In [96]:
y_pred = model.predict(x_test)
y_pred

array([[2.52036330e-12, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [2.31237326e-12, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [1.60000000e+01, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [6.20800000e+03, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [6.22400000e+03, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [6.22400000e+03, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]], shape=(780, 273))

In [97]:
np.mean(np.abs(y_test - y_pred))

np.float64(0.017667929389749332)

In [98]:
with lzma.open("../models/scout.pkl.xz", "wb") as f:
    pickle.dump(model, f)

In [99]:
class Encoder(nn.Module):
    def __init__(self, input_size, hidden_size, dropout_p=0.1):
        super().__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Linear(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, batch_first=True)
        self.dropout = nn.Dropout(dropout_p)

    def forward(self, input):
        embedded = self.dropout(self.embedding(input))
        output, hidden = self.gru(embedded)
        return output, hidden

In [100]:
class Decoder(nn.Module):
    def __init__(self, hidden_size, output_size):
        super().__init__()
        self.embedding = nn.Linear(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, batch_first=True)
        self.out = nn.Linear(hidden_size, output_size)

    def forward(self, encoder_outputs, encoder_hidden, target_tensor=None):
        output, hidden = self.gru(encoder_outputs, encoder_hidden)
        return self.out(output), hidden, None

    # def forward(self, encoder_outputs, encoder_hidden, target_tensor=None):
    #     batch_size = encoder_outputs.size(0)
    #
    #     decoder_input = torch.zeros(batch_size, 1, 273, device=device)
    #     decoder_hidden = encoder_hidden
    #     decoder_outputs = []
    #
    #     for i in range(encoder_outputs.size(1)):
    #
    #         decoder_output, decoder_hidden  = self.forward_step(decoder_input, decoder_hidden)
    #         decoder_outputs.append(decoder_output)
    #
    #         if target_tensor is not None:
    #             # Teacher forcing: Feed the target as the next input
    #             decoder_input = target_tensor[:, i].unsqueeze(1) # Teacher forcing
    #         else:
    #             # Without teacher forcing: use its own predictions as the next input
    #             _, topi = decoder_output.topk(1)
    #             decoder_input = topi.squeeze(-1).detach()  # detach from history as input
    #
    #     decoder_outputs = torch.cat(decoder_outputs, dim=1)
    #     # decoder_outputs = F.log_softmax(decoder_outputs, dim=-1)
    #     return decoder_outputs, decoder_hidden, None # We return `None` for consistency in the training loop
    #
    # def forward_step(self, input, hidden):
    #     output = self.embedding(input)
    #     output = F.relu(output)
    #     output, hidden = self.gru(output, hidden)
    #     output = self.out(output)
    #     return output, hidden

In [101]:
def train_epoch(dataloader, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion):
    total_loss = 0
    for data in dataloader:
        input_tensor, target_tensor = data

        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()

        encoder_outputs, encoder_hidden = encoder(input_tensor)
        decoder_outputs, _, _ = decoder(encoder_outputs, encoder_hidden, target_tensor)

        loss = criterion(
            decoder_outputs,
            target_tensor,
            # decoder_outputs.view(-1, decoder_outputs.size(-1)),
            # target_tensor.view(-1)
        )
        loss.backward()

        encoder_optimizer.step()
        decoder_optimizer.step()

        total_loss += loss.item()

    return total_loss / len(dataloader)

In [102]:
def test_epoch(dataloader, encoder, decoder, criterion):
    total_loss = 0
    for data in tqdm(dataloader):
        input_tensor, target_tensor = data

        encoder_outputs, encoder_hidden = encoder(input_tensor)
        decoder_outputs, _, _ = decoder(encoder_outputs, encoder_hidden, target_tensor)

        loss = criterion(
            decoder_outputs,
            target_tensor,
            # decoder_outputs.view(-1, decoder_outputs.size(-1)),
            # target_tensor.view(-1)
        )
        loss.backward()

        total_loss += loss.item()

    return total_loss / len(dataloader)

In [103]:
dataset_train = TensorDataset(torch.Tensor(x_train[np.newaxis, ...]), torch.Tensor(y_train[np.newaxis, ...]))
dataset_test = TensorDataset(torch.Tensor(x_test[np.newaxis, ...]), torch.Tensor(y_test[np.newaxis, ...]))

In [104]:
dataset_train[-1]

(tensor([[1.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
          0.0000e+00],
         [1.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
          0.0000e+00],
         [2.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
          0.0000e+00],
         ...,
         [6.2260e+03, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
          0.0000e+00],
         [6.2270e+03, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
          0.0000e+00],
         [6.2270e+03, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
          0.0000e+00]]),
 tensor([[1.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
          0.0000e+00],
         [1.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
          0.0000e+00],
         [2.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
          0.0000e+00],
         ...,
         [6.2260e+03, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
   

In [105]:
dataloader_train = DataLoader(dataset_train, batch_size=64, shuffle=True)
dataloader_test = DataLoader(dataset_test, batch_size=64, shuffle=True)

In [106]:
def train(train_dataloader, encoder, decoder, n_epochs, learning_rate=0.001):
    time.time()

    encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate)
    criterion = nn.L1Loss()

    for _epoch in range(1, n_epochs + 1):
        loss = train_epoch(train_dataloader, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion)
        print(loss)

In [ ]:
input_size = x_train.shape[1]
output_size = y_train.shape[1]
hidden_size = 128
batch_size = 32

encoder = Encoder(input_size, hidden_size).to(device)
decoder = Decoder(hidden_size, output_size).to(device)

train(dataloader_train, encoder, decoder, 10)

In [ ]:
loss = test_epoch(dataloader_test, encoder, decoder, nn.L1Loss())
print(loss)

In [ ]:
x_test_sample, y_test_sample = dataloader_test[0]
encoder_outputs, encoder_hidden = encoder(x_test_sample)
decoder_outputs, _, _ = decoder(encoder_outputs, encoder_hidden)
decoder_outputs